# This script scrapes Python-related Common Vulnerabilities and Exposures from the National Vulnerabilities Dataset

Importing the neccesary libraries.

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup

Pulling the dedicated page for Python vulnerabilities from http://cve.mitre.org
Note: keyword = python

In [2]:
URL = "http://cve.mitre.org/cgi-bin/cvekey.cgi?keyword=python"

Requesting the URL above

In [3]:
page = requests.get(URL)

Specify a desired format for the “page” using the html parser in BeautifulSoup. This allows Python to read the various components of the page, rather than treating it as a long string.

In [4]:
formated_page = BeautifulSoup(page.text, "html.parser")

Printing the formatted page in a structured tree format (using the prettify method) for readability

In [5]:
print(formated_page.prettify())

<?xml version="1.0"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="CVE, Common Vulnerabilities and Exposures, CVE List, CVE Records, CVE ID, CVE IDs, CVE Identifier, CVE Identifiers, CVE ID number, CVE ID numbers, CVE number, CVE numbers, CVE Entry, CVE Entries, CVE name, CVEs, CVE-, Reserved but Public, RBP, CVE Numbering Authority, CVE Naming Authority, CNA, CNAs, Root CNA, Top-Level Root CNA, TLR-CNA, CNA of Last Resort, CNA-LR, Secretariat, Authorized Data Publisher, ADP, CVE Adoption, National Vulnerability Database, NVD, Common Vulnerability Scoring System, CVSS, standard, standards, vulnerability, vulnerabilities, vulnerability management, vulnerability names, vulnerability naming, vuln

A function to extract the CVE ids

In [6]:
def extract_CVE_ids(formated_page):
    CVE_ids =[]
    for table in formated_page.find_all(name="div", attrs={"id":"TableWithRules"}):
        for row in table.find_all(name="tr"):
            for column in row.find_all(name="td", attrs={"valign":"top", "nowrap":"nowrap"}):
                for ids in column.find_all(name="a"):
                    CVE_ids.append(ids.string)
    return CVE_ids

Calling the extract_CVE_ids function

In [7]:
CVE_ids_list = extract_CVE_ids(formated_page)

In [8]:
CVE_ids_list

['CVE-2021-3177',
 'CVE-2021-21241',
 'CVE-2021-21239',
 'CVE-2021-21238',
 'CVE-2021-21236',
 'CVE-2020-8897',
 'CVE-2020-8492',
 'CVE-2020-8315',
 'CVE-2020-7212',
 'CVE-2020-5741',
 'CVE-2020-5740',
 'CVE-2020-5252',
 'CVE-2020-5227',
 'CVE-2020-5215',
 'CVE-2020-35678',
 'CVE-2020-29651',
 'CVE-2020-29396',
 'CVE-2020-27783',
 'CVE-2020-27619',
 'CVE-2020-27589',
 'CVE-2020-27351',
 'CVE-2020-26943',
 'CVE-2020-26268',
 'CVE-2020-26263',
 'CVE-2020-26244',
 'CVE-2020-26222',
 'CVE-2020-26116',
 'CVE-2020-25659',
 'CVE-2020-25658',
 'CVE-2020-25489',
 'CVE-2020-24715',
 'CVE-2020-24584',
 'CVE-2020-24583',
 'CVE-2020-1747',
 'CVE-2020-16977',
 'CVE-2020-15801',
 'CVE-2020-15720',
 'CVE-2020-15523',
 'CVE-2020-15415',
 'CVE-2020-15348',
 'CVE-2020-15271',
 'CVE-2020-15251',
 'CVE-2020-15207',
 'CVE-2020-15193',
 'CVE-2020-15163',
 'CVE-2020-15142',
 'CVE-2020-15141',
 'CVE-2020-15101',
 'CVE-2020-14422',
 'CVE-2020-13948',
 'CVE-2020-13757',
 'CVE-2020-13388',
 'CVE-2020-13258',
 'CV

A function to filter record in the extract_CVE_description function. The function is passed as an argument in one of its .find_all 

In [9]:
def has_valign_but_no_wrap(tag):
    return tag.has_attr('valign') and not tag.has_attr('nowrap') and not tag.has_attr('style')

Implementation of extract_CVE_description function

Importing the regular expression module to treat the vulnerability description string

In [10]:
import re

In [11]:
def extract_CVE_description(formated_page):
    CVE_descr =[]
    for table in formated_page.find_all(name="div", attrs={"id":"TableWithRules"}):
        for row in table.find_all(name="tr"):
            for descr in row.find_all(has_valign_but_no_wrap):
                delimited_descr = re.split("\n+", descr.string)
                CVE_descr.append(delimited_descr[0])
    return CVE_descr

Calling the extract_CVE_description function

In [12]:
CVE_description_list = extract_CVE_description(formated_page)

In [15]:
CVE_description_list

['Python 3.x through 3.9.1 has a buffer overflow in PyCArg_repr in _ctypes/callproc.c, which may lead to remote code execution in certain Python applications that accept floating-point numbers as untrusted input, as demonstrated by a 1e300 argument to c_double.from_param. This occurs because sprintf is used unsafely.',
 'The Python "Flask-Security-Too" package is used for adding security features to your Flask application. It is an is a independently maintained version of Flask-Security based on the 3.0.0 version of Flask-Security. In Flask-Security-Too from version 3.3.0 and before version 3.4.5, the /login and /change endpoints can return the authenticated user\'s authentication token in response to a GET request. Since GET requests aren\'t protected with a CSRF token, this could lead to a malicious 3rd party site acquiring the authentication token. Version 3.4.5 and version 4.0.0 are patched. As a workaround, if you aren\'t using authentication tokens - you can set the SECURITY_TOKE

Building frame for the extracted data

Import required library

In [13]:
import pandas as pd

In [14]:
data_frame = pd.DataFrame(list(zip(CVE_ids_list,CVE_description_list)),columns=["CVE","vulnerability_description"])

In [15]:
data_frame.head()

,CVE,vulnerability_description
0,CVE-2021-3177,Python 3.x through 3.9.1 has a buffer overflow...
1,CVE-2021-21241,"The Python ""Flask-Security-Too"" package is use..."
2,CVE-2021-21239,PySAML2 is a pure python implementation of SAM...
3,CVE-2021-21238,PySAML2 is a pure python implementation of SAM...
4,CVE-2021-21236,CairoSVG is a Python (pypi) package. CairoSVG ...


In [15]:
data_frame.to_csv(r"C:\Users\Semiu\Documents\python-codesecurity\data\pythonVulData.csv", encoding='utf-8')

This is when the CVE and their respective descriptions are expected to be stored in a database.


In [16]:
data_dictionary = data_frame.to_dict('records')

In [17]:
data_dictionary

[{'CVE': 'CVE-2021-3177',
  'vulnerability_description': 'Python 3.x through 3.9.1 has a buffer overflow in PyCArg_repr in _ctypes/callproc.c, which may lead to remote code execution in certain Python applications that accept floating-point numbers as untrusted input, as demonstrated by a 1e300 argument to c_double.from_param. This occurs because sprintf is used unsafely.'},
 {'CVE': 'CVE-2021-21241',
  'vulnerability_description': 'The Python "Flask-Security-Too" package is used for adding security features to your Flask application. It is an is a independently maintained version of Flask-Security based on the 3.0.0 version of Flask-Security. In Flask-Security-Too from version 3.3.0 and before version 3.4.5, the /login and /change endpoints can return the authenticated user\'s authentication token in response to a GET request. Since GET requests aren\'t protected with a CSRF token, this could lead to a malicious 3rd party site acquiring the authentication token. Version 3.4.5 and vers

Import required modules 

In [18]:
import pymysql

In [21]:
connection = pymysql.connect(host='127.0.0.1', user='root', passwd="AYObami@1984", db='mysql')

In [22]:
cursor = connection.cursor()

In [23]:
cursor.execute('USE vuldb')

0

In [24]:
def store_to_db(cve, descr):
    cursor.execute('INSERT INTO PYTHON_CVEVUL_TBL (cve_name, cve_description) VALUES''("%s", "%s")', (cve, descr))
    cursor.connection.commit()
    

In [25]:
def from_dict_to_db (data_dictionary):
    for items in data_dictionary:
        if 'CVE' in items and 'vulnerability_description' in items:
            #calling the store_to_db function
            store_to_db(items['CVE'], items['vulnerability_description'])

In [26]:
from_dict_to_db (data_dictionary)

Close the cursor and connection

In [27]:
cursor.close()

In [28]:
connection.close()